In [1]:
#RR
import pandas as pd

class Process:
    def __init__(self, PID, arrival_time, burst_time):
        self.PID = PID
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.waiting_time = 0
        self.remaining_time = burst_time
        self.complete_time = 0
        self.turn_around_time = 0
        self.start_time = -1

def round_robin_scheduler(file_path, quantum_time=12):
    # Baca data dari Excel
    df = pd.read_excel(file_path)

    # Inisialisasi proses
    processes = []
    for i in range(len(df)):
        processes.append(Process(df['PID'][i], df['arrival_time'][i], df['Burst_time'][i]))

    # Sort processes berdasarkan arrival time awal
    processes.sort(key=lambda x: x.arrival_time)

    time = 0
    queue = []
    running = None
    finished_jobs = 0
    remaining_quantum = quantum_time

    while finished_jobs < len(processes):
        # Cek proses yang baru datang
        for process in processes:
            if process.arrival_time == time:
                queue.append(process)

        # Jika tidak ada proses yang running, ambil dari queue
        if running is None and queue:
            running = queue.pop(0)
            if running.start_time == -1:
                running.start_time = time
            remaining_quantum = quantum_time

        # Eksekusi proses yang sedang running
        if running is not None:
            running.remaining_time -= 1
            remaining_quantum -= 1

            # Jika proses selesai
            if running.remaining_time == 0:
                running.complete_time = time + 1
                running.turn_around_time = running.complete_time - running.arrival_time
                running.waiting_time = running.turn_around_time - running.burst_time
                finished_jobs += 1
                running = None

            # Jika quantum habis tapi proses belum selesai
            elif remaining_quantum == 0:
                queue.append(running)
                running = None

        # Update waiting time untuk proses di queue
        for process in queue:
            process.waiting_time += 1
            process.turn_around_time += 1

        time += 1

        # Jika running kosong dan masih ada proses di queue, ambil proses berikutnya
        if running is None and queue:
            running = queue.pop(0)
            if running.start_time == -1:
                running.start_time = time
            remaining_quantum = quantum_time

    # Menyiapkan hasil dan sort berdasarkan start time
    results = []
    total_waiting_time = 0
    total_turnaround_time = 0

    for process in processes:
        results.append({
            'PID': process.PID,
            'Arrival Time': process.arrival_time,
            'Burst Time': process.burst_time,
            'Start Time': process.start_time,
            'Completion Time': process.complete_time,
            'Waiting Time': process.waiting_time,
            'Turnaround Time': process.turn_around_time
        })
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turn_around_time

    # Sort berdasarkan start time
    results.sort(key=lambda x: x['Start Time'])

    avg_waiting_time = total_waiting_time / len(processes)
    avg_turnaround_time = total_turnaround_time / len(processes)

    return results, avg_waiting_time, avg_turnaround_time

def print_results(results, avg_waiting_time, avg_turnaround_time):
    print("\n" + "="*100)
    print(f"Hasil Penjadwalan Round Robin (Sorted by Start Time)")
    print("="*100)

    # Print table header
    print(f"{'PID':^10} | {'Arrival':^10} | {'Burst':^10} | {'Start':^10} | {'Complete':^10} | {'Waiting':^10} | {'Turnaround':^10}")
    print("-"*100)

    # Print data
    for result in results:
        print(f"{result['PID']:^10} | {result['Arrival Time']:^10} | {result['Burst Time']:^10} | "
              f"{result['Start Time']:^10} | {result['Completion Time']:^10} | {result['Waiting Time']:^10} | "
              f"{result['Turnaround Time']:^10}")

    print("-"*100)
    print(f"Average Waiting Time: {avg_waiting_time:.2f}")
    print(f"Average Turnaround Time: {avg_turnaround_time:.2f}")
    print("="*100)

# Penggunaan
file_path = '/content/sample_data/processes.xlsx'
results, avg_waiting_time, avg_turnaround_time = round_robin_scheduler(file_path)
print_results(results, avg_waiting_time, avg_turnaround_time)


Hasil Penjadwalan Round Robin (Sorted by Start Time)
   PID     |  Arrival   |   Burst    |   Start    |  Complete  |  Waiting   | Turnaround
----------------------------------------------------------------------------------------------------
   P51     |     0      |     15     |     0      |    346     |    331     |    346    
   P66     |     0      |     14     |     12     |    563     |    549     |    563    
   P98     |     0      |     14     |     24     |    750     |    736     |    750    
   P39     |     1      |     19     |     36     |    963     |    943     |    962    
   P40     |     1      |     9      |     48     |     57     |     47     |     56    
   P90     |     1      |     17     |     57     |    1063    |    1045    |    1062   
   P23     |     2      |     16     |     69     |    1067    |    1049    |    1065   
   P55     |     2      |     11     |     81     |     92     |     79     |     90    
    P8     |     3      |     7      |     9

In [ ]:
#FCFS
import pandas as pd

class Process:
    def __init__(self, PID, arrival_time, burst_time):
        self.PID = PID
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.waiting_time = 0
        self.complete_time = 0
        self.turn_around_time = 0
        self.start_time = 0

file_path = '/content/sample_data/processes.xlsx'
df = pd.read_excel(file_path)

processes = []
for i in range(len(df)):
    processes.append(Process(df['PID'][i], df['arrival_time'][i], df['Burst_time'][i]))
processes.sort(key=lambda x: x.arrival_time)

current_time = 0
resultsFCFS = []

for process in processes:
    if current_time < process.arrival_time:
        current_time = process.arrival_time

    process.start_time = current_time
    process.complete_time = current_time + process.burst_time
    process.waiting_time = process.start_time - process.arrival_time
    process.turn_around_time = process.waiting_time + process.burst_time

    current_time = process.complete_time

    resultsFCFS.append({
        'PID': process.PID,
        'Arrival Time': process.arrival_time,
        'Burst Time': process.burst_time,
        'Start Time': process.start_time,
        'Completion Time': process.complete_time,
        'Waiting Time': process.waiting_time,
        'Turnaround Time': process.turn_around_time
    })

total_waiting_time = sum(p['Waiting Time'] for p in resultsFCFS)
total_turnaround_time = sum(p['Turnaround Time'] for p in resultsFCFS)
average_waiting_time = total_waiting_time / len(processes)
average_turnaround_time = total_turnaround_time / len(processes)

print("Hasil Eksekusi Proses FCFS:")
print("-" * 80)
for result in results:
        print(f"{result['PID']:^10} | {result['Arrival Time']:^10} | {result['Burst Time']:^10} | "
              f"{result['Start Time']:^10} | {result['Completion Time']:^10} | {result['Waiting Time']:^10} | "
              f"{result['Turnaround Time']:^10}")

print(f"\nRata-rata Waiting Time: {average_waiting_time:.2f}")
print(f"Rata-rata Turnaround Time: {average_turnaround_time:.2f}")

Hasil Eksekusi Proses FCFS:
--------------------------------------------------------------------------------
   P51     |     0      |     15     |     0      |    346     |    331     |    346    
   P66     |     0      |     14     |     12     |    563     |    549     |    563    
   P98     |     0      |     14     |     24     |    750     |    736     |    750    
   P39     |     1      |     19     |     36     |    963     |    943     |    962    
   P40     |     1      |     9      |     48     |     57     |     47     |     56    
   P90     |     1      |     17     |     57     |    1063    |    1045    |    1062   
   P23     |     2      |     16     |     69     |    1067    |    1049    |    1065   
   P55     |     2      |     11     |     81     |     92     |     79     |     90    
    P8     |     3      |     7      |     92     |     99     |     89     |     96    
   P21     |     4      |     6      |     99     |    105     |     95     |    101    
 

In [ ]:
#SJF Non-Preemtive
import pandas as pd

class Process:
    def __init__(self, PID, arrival_time, burst_time):
        self.PID = PID
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.start_time = 0
        self.complete_time = 0
        self.waiting_time = 0
        self.turn_around_time = 0

def sjf_non_preemptive_scheduler(file_path):
    # Baca data dari Excel
    df = pd.read_excel(file_path)

    # Inisialisasi proses
    processes = []
    for i in range(len(df)):
        processes.append(Process(df['PID'][i], df['arrival_time'][i], df['Burst_time'][i]))

    n = len(processes)
    completed = 0
    current_time = 0
    is_completed = [0] * n

    while completed != n:
        # Cari proses dengan burst time terpendek yang sudah arrival
        eligible_process = float('inf')
        selected_process = -1

        for i in range(n):
            if processes[i].arrival_time <= current_time and is_completed[i] == 0:
                if processes[i].burst_time < eligible_process:
                    eligible_process = processes[i].burst_time
                    selected_process = i
                # Jika burst time sama, pilih yang arrival time lebih awal
                elif processes[i].burst_time == eligible_process:
                    if processes[i].arrival_time < processes[selected_process].arrival_time:
                        selected_process = i

        if selected_process == -1:
            # Jika tidak ada proses yang eligible, cari arrival time terdekat
            next_arrival = float('inf')
            for i in range(n):
                if is_completed[i] == 0 and processes[i].arrival_time < next_arrival:
                    next_arrival = processes[i].arrival_time
            current_time = next_arrival
        else:
            # Eksekusi proses yang terpilih
            process = processes[selected_process]
            process.start_time = current_time
            process.complete_time = current_time + process.burst_time
            process.turn_around_time = process.complete_time - process.arrival_time
            process.waiting_time = process.turn_around_time - process.burst_time

            is_completed[selected_process] = 1
            completed += 1
            current_time = process.complete_time

    # Menyiapkan hasil
    results = []
    total_waiting_time = 0
    total_turnaround_time = 0

    for process in processes:
        results.append({
            'PID': process.PID,
            'Arrival Time': process.arrival_time,
            'Burst Time': process.burst_time,
            'Start Time': process.start_time,
            'Completion Time': process.complete_time,
            'Waiting Time': process.waiting_time,
            'Turnaround Time': process.turn_around_time
        })
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turn_around_time

    # Sort berdasarkan start time
    results.sort(key=lambda x: x['Start Time'])

    return results, total_waiting_time/n, total_turnaround_time/n

def print_results(results, avg_waiting_time, avg_turnaround_time):
    print("\n" + "="*100)
    print("Hasil Penjadwalan SJF Non-Preemptive (Sorted by Start Time)")
    print("="*100)

    # Print table header
    print(f"{'PID':^10} | {'Arrival':^10} | {'Burst':^10} | {'Start':^10} | {'Complete':^10} | {'Waiting':^10} | {'Turnaround':^10}")
    print("-"*100)

    # Print data
    for result in results:
        print(f"{result['PID']:^10} | {result['Arrival Time']:^10} | {result['Burst Time']:^10} | "
              f"{result['Start Time']:^10} | {result['Completion Time']:^10} | {result['Waiting Time']:^10} | "
              f"{result['Turnaround Time']:^10}")

    print("-"*100)
    print(f"Average Waiting Time: {avg_waiting_time:.2f}")
    print(f"Average Turnaround Time: {avg_turnaround_time:.2f}")
    print("="*100)

# Penggunaan
file_path = '/content/sample_data/processes.xlsx'
results, avg_waiting_time, avg_turnaround_time = sjf_non_preemptive_scheduler(file_path)
print_results(results, avg_waiting_time, avg_turnaround_time)


Hasil Penjadwalan SJF Non-Preemptive (Sorted by Start Time)
   PID     |  Arrival   |   Burst    |   Start    |  Complete  |  Waiting   | Turnaround
----------------------------------------------------------------------------------------------------
   P66     |     0      |     14     |     0      |     14     |     0      |     14    
   P88     |     5      |     5      |     14     |     19     |     9      |     14    
   P85     |     10     |     5      |     19     |     24     |     9      |     14    
   P28     |     13     |     5      |     24     |     29     |     11     |     16    
   P86     |     21     |     5      |     29     |     34     |     8      |     13    
   P21     |     4      |     6      |     34     |     40     |     30     |     36    
   P25     |     39     |     5      |     40     |     45     |     1      |     6     
   P34     |     8      |     6      |     45     |     51     |     37     |     43    
   P57     |     15     |     6      

In [ ]:
#SJF Preemtive
import pandas as pd

class Process:
    def __init__(self, PID, arrival_time, burst_time):  # Memperbaiki nama konstruktor
        self.PID = PID
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.start_time = -1
        self.complete_time = 0
        self.waiting_time = 0
        self.turn_around_time = 0

def schedule_processes(file_path):
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print("File tidak ditemukan")
        return

    processes = []
    for i in range(len(df)):
        processes.append(Process(df['PID'][i], df['arrival_time'][i], df['Burst_time'][i]))

    time = 0
    queue = []
    running = None
    finished_jobs = 0

    while finished_jobs < len(processes):
        # Check for new arrivals
        for process in processes:
            if process.arrival_time == time:
                queue.append(process)

        # Check if we need to switch processes
        if running is None or (running.remaining_time > 0 and any(q.remaining_time < running.remaining_time for q in queue)):
            shortest_process = None
            for process in queue:
                if process.remaining_time > 0:
                    if shortest_process is None or process.remaining_time < shortest_process.remaining_time:
                        shortest_process = process
                    elif process.remaining_time == shortest_process.remaining_time and process.arrival_time < shortest_process.arrival_time:
                        shortest_process = process

            if running is not None and running.remaining_time > 0:
                queue.append(running)

            if shortest_process is not None:
                queue.remove(shortest_process)
                running = shortest_process
                if running.start_time == -1:
                    running.start_time = time

        # Process the current running process
        if running is not None:
            running.remaining_time -= 1
            if running.remaining_time == 0:
                running.complete_time = time + 1
                running.turn_around_time = running.complete_time - running.arrival_time
                running.waiting_time = running.turn_around_time - running.burst_time
                finished_jobs += 1
                running = None

        # Update waiting time for processes in queue
        for process in queue:
            if process != running and process.remaining_time > 0:
                process.waiting_time += 1

        time += 1

    # Create results and sort by start_time
    results = []
    total_waiting_time = 0
    total_turnaround_time = 0

    for process in processes:
        results.append({
            'PID': process.PID,
            'Arrival Time': process.arrival_time,
            'Burst Time': process.burst_time,
            'Start Time': process.start_time,
            'Completion Time': process.complete_time,
            'Waiting Time': process.waiting_time,
            'Turnaround Time': process.turn_around_time
        })
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turn_around_time

    # Sort results by start time
    results.sort(key=lambda x: x['Start Time'])

    # Print sorted results
    print("\nProses terurut berdasarkan Start Time:")
    print("-" * 80)
    print(f"{'PID':<6} {'Arrival':<8} {'Burst':<7} {'Start':<7} {'Completion':<12} {'Waiting':<9} {'Turnaround':<10}")
    print("-" * 80)

    for result in results:
        print(f"{result['PID']:<6} {result['Arrival Time']:<8} {result['Burst Time']:<7} "
              f"{result['Start Time']:<7} {result['Completion Time']:<12} {result['Waiting Time']:<9} "
              f"{result['Turnaround Time']:<10}")

    print("-" * 80)
    average_waiting_time = total_waiting_time / len(processes)
    average_turnaround_time = total_turnaround_time / len(processes)
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

# Penggunaan
file_path = '/content/sample_data/processes.xlsx'
schedule_processes(file_path)



Proses terurut berdasarkan Start Time:
--------------------------------------------------------------------------------
PID    Arrival  Burst   Start   Completion   Waiting   Turnaround
--------------------------------------------------------------------------------
P66    0        14      0       422          408       422       
P40    1        9       1       10           0         9         
P88    5        5       10      15           5         10        
P85    10       5       15      20           5         10        
P28    13       5       20      25           7         12        
P86    21       5       25      30           4         9         
P21    4        6       30      36           26        32        
P34    8        6       36      42           28        34        
P25    39       5       42      47           3         8         
P57    15       6       47      53           32        38        
P49    17       6       53      59           36        42        
P70   

In [ ]:
#LJF Preemtive
import pandas as pd

class Process:
    def __init__(self, PID, arrival_time, burst_time):
        self.PID = PID
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.start_time = -1
        self.complete_time = 0
        self.waiting_time = 0
        self.turn_around_time = 0

def schedule_processes(file_path):
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print("File tidak ditemukan")
        return

    processes = []
    for i in range(len(df)):
        processes.append(Process(df['PID'][i], df['arrival_time'][i], df['Burst_time'][i]))

    time = 0
    queue = []
    running = None
    finished_jobs = 0

    while finished_jobs < len(processes):
        # Add processes that arrive at the current time
        for process in processes:
            if process.arrival_time == time:
                queue.append(process)

        # If no process is running or we need to choose a new process based on LJF
        if running is None or (running.remaining_time > 0 and any(q.remaining_time > running.remaining_time for q in queue)):
            longest_process = None
            # Find the process with the longest burst time that has arrived
            for process in queue:
                if process.remaining_time > 0:
                    if longest_process is None or process.remaining_time > longest_process.remaining_time:
                        longest_process = process
                    # If burst times are the same, choose the one with the earlier arrival
                    elif process.remaining_time == longest_process.remaining_time and process.arrival_time < longest_process.arrival_time:
                        longest_process = process

            if longest_process is not None:
                # If we are already running a process, and we choose a new one, preempt it
                if running is not None and running != longest_process:
                    queue.append(running)  # Put the preempted process back into the queue

                running = longest_process
                queue.remove(longest_process)

                if running.start_time == -1:
                    running.start_time = time

        # Process the current running process
        if running is not None:
            running.remaining_time -= 1
            if running.remaining_time == 0:
                running.complete_time = time + 1
                running.turn_around_time = running.complete_time - running.arrival_time
                running.waiting_time = running.turn_around_time - running.burst_time
                finished_jobs += 1
                running = None

        # Update waiting time for processes in queue
        for process in queue:
            if process != running and process.remaining_time > 0:
                process.waiting_time += 1

        time += 1

    # Create results and sort by start_time
    results = []
    total_waiting_time = 0
    total_turnaround_time = 0

    for process in processes:
        results.append({
            'PID': process.PID,
            'Arrival Time': process.arrival_time,
            'Burst Time': process.burst_time,
            'Start Time': process.start_time,
            'Completion Time': process.complete_time,
            'Waiting Time': process.waiting_time,
            'Turnaround Time': process.turn_around_time
        })
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turn_around_time

    # Sort results by start time
    results.sort(key=lambda x: x['Start Time'])

    # Print sorted results
    print("\nProses terurut berdasarkan Start Time:")
    print("-" * 80)
    print(f"{'PID':<6} {'Arrival':<8} {'Burst':<7} {'Start':<7} {'Completion':<12} {'Waiting':<9} {'Turnaround':<10}")
    print("-" * 80)

    for result in results:
        print(f"{result['PID']:<6} {result['Arrival Time']:<8} {result['Burst Time']:<7} "
              f"{result['Start Time']:<7} {result['Completion Time']:<12} {result['Waiting Time']:<9} "
              f"{result['Turnaround Time']:<10}")

    print("-" * 80)
    average_waiting_time = total_waiting_time / len(processes)
    average_turnaround_time = total_turnaround_time / len(processes)
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

# Penggunaan
file_path = '/content/sample_data/processes.xlsx'
schedule_processes(file_path)



Proses terurut berdasarkan Start Time:
--------------------------------------------------------------------------------
PID    Arrival  Burst   Start   Completion   Waiting   Turnaround
--------------------------------------------------------------------------------
P51    0        15      0       1162         1147      1162      
P39    1        19      1       1164         1144      1163      
P90    1        17      4       1165         1147      1164      
P44    4        17      5       1170         1149      1166      
P60    6        18      6       1176         1152      1170      
P16    10       20      10      1191         1161      1181      
P61    13       19      13      1196         1164      1183      
P75    15       19      15      1201         1167      1186      
P52    16       20      16      1203         1167      1187      
P43    10       18      19      1192         1164      1182      
P91    20       19      20      1210         1171      1190      
P96   

In [ ]:
#LJF Non-Preemtive
import pandas as pd

class Process:
    def __init__(self, PID, arrival_time, burst_time):
        self.PID = PID
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.start_time = -1
        self.complete_time = 0
        self.waiting_time = 0
        self.turn_around_time = 0

def schedule_processes(file_path):
    try:
        df = pd.read_excel(file_path)
    except FileNotFoundError:
        print("File tidak ditemukan")
        return

    processes = []
    for i in range(len(df)):
        processes.append(Process(df['PID'][i], df['arrival_time'][i], df['Burst_time'][i]))

    time = 0
    queue = []
    running = None
    finished_jobs = 0

    while finished_jobs < len(processes):
        # Add processes that arrive at the current time
        for process in processes:
            if process.arrival_time == time:
                queue.append(process)

        # If no process is running or we need to choose a new process based on LJF
        if running is None:
            longest_process = None
            # Find the process with the longest burst time that has arrived
            for process in queue:
                if process.remaining_time > 0:
                    if longest_process is None or process.remaining_time > longest_process.remaining_time:
                        longest_process = process
                    # If burst times are the same, choose the one with the earlier arrival
                    elif process.remaining_time == longest_process.remaining_time and process.arrival_time < longest_process.arrival_time:
                        longest_process = process

            if longest_process is not None:
                running = longest_process
                queue.remove(longest_process)
                if running.start_time == -1:
                    running.start_time = time

        # Process the current running process
        if running is not None:
            running.remaining_time -= 1
            if running.remaining_time == 0:
                running.complete_time = time + 1
                running.turn_around_time = running.complete_time - running.arrival_time
                running.waiting_time = running.turn_around_time - running.burst_time
                finished_jobs += 1
                running = None

        # Update waiting time for processes in queue
        for process in queue:
            if process != running and process.remaining_time > 0:
                process.waiting_time += 1

        time += 1

    # Create results and sort by start_time
    results = []
    total_waiting_time = 0
    total_turnaround_time = 0

    for process in processes:
        results.append({
            'PID': process.PID,
            'Arrival Time': process.arrival_time,
            'Burst Time': process.burst_time,
            'Start Time': process.start_time,
            'Completion Time': process.complete_time,
            'Waiting Time': process.waiting_time,
            'Turnaround Time': process.turn_around_time
        })
        total_waiting_time += process.waiting_time
        total_turnaround_time += process.turn_around_time

    # Sort results by start time
    results.sort(key=lambda x: x['Start Time'])

    # Print sorted results
    print("\nProses terurut berdasarkan Start Time:")
    print("-" * 80)
    print(f"{'PID':<6} {'Arrival':<8} {'Burst':<7} {'Start':<7} {'Completion':<12} {'Waiting':<9} {'Turnaround':<10}")
    print("-" * 80)

    for result in results:
        print(f"{result['PID']:<6} {result['Arrival Time']:<8} {result['Burst Time']:<7} "
              f"{result['Start Time']:<7} {result['Completion Time']:<12} {result['Waiting Time']:<9} "
              f"{result['Turnaround Time']:<10}")

    print("-" * 80)
    average_waiting_time = total_waiting_time / len(processes)
    average_turnaround_time = total_turnaround_time / len(processes)
    print(f"\nAverage Waiting Time: {average_waiting_time:.2f}")
    print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

# Penggunaan
file_path = '/content/sample_data/processes.xlsx'
schedule_processes(file_path)



Proses terurut berdasarkan Start Time:
--------------------------------------------------------------------------------
PID    Arrival  Burst   Start   Completion   Waiting   Turnaround
--------------------------------------------------------------------------------
P51    0        15      0       15           0         15        
P16    10       20      15      35           5         25        
P52    16       20      35      55           19        39        
P53    34       20      55      75           21        41        
P33    37       20      75      95           38        58        
P22    42       20      95      115          53        73        
P39    1        19      115     134          114       133       
P61    13       19      134     153          121       140       
P75    15       19      153     172          138       157       
P91    20       19      172     191          152       171       
P37    34       19      191     210          157       176       
P89   